In [591]:
from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES
import json
import time
import random
import string
import random
import string
from Crypto.Hash import SHA256

In [592]:
sent_messages = []

def hash_value(data):
    h = SHA256.new()
    h.update(data)
    return h.hexdigest()


def xor_bytes(bytes1, bytes2):
    return bytes([b1 ^ b2 for b1, b2 in zip(bytes1, bytes2)])


def get_timestamp():
    return int(time.time())

def send_message(message):
    json_message = json.dumps(message)
    print("Sending message:", json_message)
    sent_messages.append(message)

In [593]:
serial_numbers = []
session_keys = {}
multicast_groups = {}
num_smart_meters = 3
gateway_key = "".join(
    random.choice(string.ascii_letters + string.digits) for _ in range(16)
).encode()


def generate_serial_number(length=10):
    characters = string.digits
    serial_number = "".join(random.choice(characters) for _ in range(length))
    return serial_number


def generate_smart_meter(smart_meter_id, gateway_key, serial_number):

    h_sn = hash_value(serial_number.encode())
    x_i = get_random_bytes(16)
    n_i = xor_bytes(
        bytes.fromhex(hash_value((smart_meter_id + h_sn).encode())),
        bytes.fromhex(hash_value(gateway_key + x_i)),
    )
    return {
        "h_sn": h_sn,
        "n_i": n_i.hex(),
        "x_i": x_i.hex(),
        "serial_number": serial_number,
    }


def generate_smart_meters(num_smart_meters, gateway_key):
    smart_meter_ids = []
    serial_numbers =[]
    registration_info = {}
    
    for i in range(num_smart_meters):
        smart_meter_id = f"SM{i+1}"
        serial_number = generate_serial_number()
        serial_numbers.append(serial_number)
        smart_meter_ids.append(smart_meter_id)
        
        registration_info[smart_meter_id] = generate_smart_meter(
            smart_meter_id, gateway_key, serial_number
        )
    return smart_meter_ids, registration_info , serial_numbers


smart_meter_ids, registration_info , serial_numbers = generate_smart_meters(
    num_smart_meters, gateway_key
)

print("Gateway Key:")
print(f" {gateway_key.decode('utf-8')}")

print("\nSmart Meter Registration Information:")
for smart_meter_id, info in registration_info.items():
    print(f"Smart Meter ID: {smart_meter_id}")
    print(f" Serial Number: {info['serial_number']}")
    print(f" H_SN: {info['h_sn']}")
    print(f" N_I: {info['n_i']}")
    print(f" X_I: {info['x_i']}")
    print()

print("Smart Meter IDs:")
for i, smart_meter_id in enumerate(smart_meter_ids):
    print(f" {i+1}. {smart_meter_id}")

Gateway Key:
 mqw8NPfyy3LVHErZ

Smart Meter Registration Information:
Smart Meter ID: SM1
 Serial Number: 8699329568
 H_SN: 33fe386e5930852f437d79a30856c8f5087b243f7daa391a0889f195a23aa9ff
 N_I: 890f1a273b2bbe1cecd580e97640e4085cb29fc7fb8a2cf49c1adbe17065cef1
 X_I: 828b52b60a9461ded3f73e55f2bfcf0d

Smart Meter ID: SM2
 Serial Number: 4270377773
 H_SN: 1235a7c46fb76cd78e631327e23f273e524db3c81f5e3bb1a8f2887f49fefe5e
 N_I: 99bd2e3180a3dd4fc748792f31b1930a8eb851245da21740e65854ff75fba173
 X_I: d50cea85973132b9e49a0830e132f3dc

Smart Meter ID: SM3
 Serial Number: 3584249911
 H_SN: 43d054a780a6416f01e81b2ba086b47933ebb67e36a1fbda52b509583595af81
 N_I: c8f49a8af4308c2377bde3564319410b6f4f1cf05952e245c2e78e9b4df609cd
 X_I: e0679660397764561649b098488701ae

Smart Meter IDs:
 1. SM1
 2. SM2
 3. SM3


In [594]:
def smart_meter_registration(smart_meter_id, serial_number):
    h_sn = hash_value(serial_number.encode())
    message = {"smart_meter_id": smart_meter_id, "h_sn": h_sn}
    send_message(message)


In [595]:
def gateway_registration_processing(message, gateway_key):
    smart_meter_id = message["smart_meter_id"]
    h_sn = message["h_sn"]
    x_i = get_random_bytes(16)
    n_i = xor_bytes(
        bytes.fromhex(hash_value((smart_meter_id + h_sn).encode())),
        bytes.fromhex(hash_value(gateway_key + x_i)),
    )

    registration_info = {
        "smart_meter_id": smart_meter_id,
        "n_i": n_i.hex(),
        "x_i": x_i.hex(),
    }
    
    print(f"\nRegistering: {smart_meter_id}")
    print(f" N_I: {registration_info['n_i']}")
    print(f" X_I: {registration_info['x_i']}")
    print()
    
    response_message = {
        "smart_meter_id": smart_meter_id,
        "n_i": n_i.hex(),
        "x_i": x_i.hex(),
        "hh_sn_i": hash_value((h_sn).encode()),
    }
    send_message(response_message)


def smart_meter_authentication(smart_meter_id, h_sn, n_i, x_i, gateway_key):
    timestamp = get_timestamp()
    
    did_i = xor_bytes(
        bytes.fromhex(hash_value((smart_meter_id + h_sn).encode())),
        bytes.fromhex(hash_value((x_i.hex() + str(timestamp)).encode())),
    )
    k_i = get_random_bytes(16)
    pk_i = hash_value(
        xor_bytes(
            bytes.fromhex(
                hash_value((gateway_key + x_i).hex().encode() + str(timestamp).encode())
            ),
            k_i,
        )
    )
    b_i = hash_value((did_i.hex() + x_i.hex() + str(timestamp) + pk_i).encode())
    
    message = {
        "smart_meter_id": smart_meter_id,
        "did_i": did_i.hex(),
        "b_i": b_i,
        "timestamp": timestamp,
        "pk_i": pk_i,
    }
    send_message(message)

In [596]:
def gateway_authentication(message, gateway_key, registration_info):
    smart_meter_id = message["smart_meter_id"]
    did_i = message["did_i"]
    b_i = message["b_i"]
    timestamp = message["timestamp"]
    pk_i = message["pk_i"]
    h_sn = registration_info[smart_meter_id]["h_sn"]

    x_i = bytes.fromhex(registration_info[smart_meter_id]["x_i"])
    current_timestamp = get_timestamp()

    if abs(current_timestamp - timestamp) > 300:
        print("Timestamp verification failed.")
        return
# TODO: add hash
    expected_b_i = hash_value(
        (
            xor_bytes(
                xor_bytes(
                    bytes.fromhex(did_i),
                    bytes.fromhex(hash_value((x_i.hex() + str(timestamp)).encode()))
                ),
                x_i
            ) +
            str(timestamp).encode() +
            pk_i.encode()
        )
    )
    if b_i != expected_b_i:
        print("B_i verification failed.")
        return

    k_i = xor_bytes(
        bytes.fromhex(
            hash_value((gateway_key + x_i).hex().encode() + str(timestamp).encode())
        ),
        bytes.fromhex(pk_i),
    )

    k_gw = get_random_bytes(16)
    pk_gw = xor_bytes(
        k_gw, bytes.fromhex(hash_value(k_i + str(current_timestamp).encode() + x_i))
    ).hex()
# TODO: add xor
    c_i = hash_value(
        did_i.encode()
        + str(current_timestamp).encode()
        + k_i.hex().encode()
        + pk_gw.encode()
    ) + hash_value(gateway_key + x_i)

    response_message = {
        "smart_meter_id": smart_meter_id,
        "c_i": c_i,
        "timestamp": current_timestamp,
        "pk_gw": pk_gw,
    }
    send_message(response_message)

    session_key = hash_value(k_i + k_gw)
    print("Shared session key for", smart_meter_id + ":", session_key)

In [597]:
def key_refreshment(smart_meter_id, session_key, gateway_key, x_i):
    new_session_key = hash_value(
        xor_bytes(
            bytes.fromhex(session_key), bytes.fromhex(hash_value((gateway_key + x_i)))
        )
    )
    print("New session key for", smart_meter_id + ":", new_session_key)
    session_keys[smart_meter_id] = new_session_key
    smart_meter_authentication(
        smart_meter_id,
        registration_info[smart_meter_id]["h_sn"],
        registration_info[smart_meter_id]["n_i"],
        bytes.fromhex(registration_info[smart_meter_id]["x_i"]),
        gateway_key,
    )

In [598]:
def establish_multicast_group(group_id, smart_meter_ids, gateway_key):
    group_key = get_random_bytes(16)
    for smart_meter_id in smart_meter_ids:
        x_a = get_random_bytes(16)
        session_key = bytes.fromhex(session_keys[smart_meter_id])
        cipher = AES.new(session_key, AES.MODE_EAX)
        nonce = cipher.nonce
        ciphertext, tag = cipher.encrypt_and_digest(
            json.dumps(
                {
                    "group_id": group_id,
                    "timestamp": get_timestamp(),
                    "x_a": x_a.hex(),
                    "group_key": group_key.hex(),
                }
            ).encode()
        )
        message = {
            "smart_meter_id": smart_meter_id,
            "timestamp": get_timestamp(),
            "encrypted_data": (nonce + tag + ciphertext).hex(),
        }
        send_message(message)

In [599]:
def join_multicast_group(message, gateway_key):
    smart_meter_id = message["smart_meter_id"]
    timestamp = message["timestamp"]
    encrypted_data = bytes.fromhex(message["encrypted_data"])
    nonce = encrypted_data[:16]
    tag = encrypted_data[16:32]
    ciphertext = encrypted_data[32:]
    session_key = bytes.fromhex(session_keys[smart_meter_id])
    cipher = AES.new(session_key, AES.MODE_EAX, nonce=nonce)
    try:
        plaintext = cipher.decrypt_and_verify(ciphertext, tag)
        multicast_info = json.loads(plaintext.decode())
        group_id = multicast_info["group_id"]
        x_a = bytes.fromhex(multicast_info["x_a"])
        group_key = bytes.fromhex(multicast_info["group_key"])
        response_message = {
            "smart_meter_id": smart_meter_id,
            "timestamp": get_timestamp(),
            "encrypted_data": ciphertext.hex(),
        }
        send_message(response_message)
        multicast_groups[group_id] = {
            "group_key": group_key,
            "members": [smart_meter_id],
        }
        print("Smart meter", smart_meter_id, "joined multicast group", group_id)
    except (ValueError, KeyError) as e:
        print(
            "Error occurred during multicast group join for smart meter", smart_meter_id
        )
        print("Error message:", str(e))

In [600]:
for i in range(len(smart_meter_ids)):
    smart_meter_registration(smart_meter_ids[i], serial_numbers[i])
for i in range(len(smart_meter_ids)):
    registration_request = {
        "smart_meter_id": smart_meter_ids[i],
        "h_sn": hash_value(serial_numbers[i].encode()),
    }
    gateway_registration_processing(registration_request, gateway_key)
for i in range(len(smart_meter_ids)):
    smart_meter_authentication(
        smart_meter_ids[i],
        registration_info[smart_meter_ids[i]]["h_sn"],
        registration_info[smart_meter_ids[i]]["n_i"],
        bytes.fromhex(registration_info[smart_meter_ids[i]]["x_i"]),
        gateway_key,
    )
    session_keys[smart_meter_ids[i]] = hash_value(
        get_random_bytes(16) + get_random_bytes(16)
    )
for i in range(len(smart_meter_ids)):
    message = {
        "smart_meter_id": smart_meter_ids[i],
        "did_i": hash_value(
            (
                smart_meter_ids[i]
                + registration_info[smart_meter_ids[i]]["h_sn"]
                + registration_info[smart_meter_ids[i]]["x_i"]
                + str(get_timestamp())
            ).encode()
        ),
        "b_i": hash_value(
            (
                hash_value(
                    (
                        smart_meter_ids[i]
                        + registration_info[smart_meter_ids[i]]["h_sn"]
                        + registration_info[smart_meter_ids[i]]["x_i"]
                        + str(get_timestamp())
                    ).encode()
                )
                + registration_info[smart_meter_ids[i]]["x_i"]
                + str(get_timestamp())
                + session_keys[smart_meter_ids[i]]
            ).encode()
        ),
        "timestamp": get_timestamp(),
        "pk_i": session_keys[smart_meter_ids[i]],
    }
    gateway_authentication(message, gateway_key, registration_info)
for i in range(len(smart_meter_ids)):
    key_refreshment(
        smart_meter_ids[i],
        session_keys[smart_meter_ids[i]],
        gateway_key,
        bytes.fromhex(registration_info[smart_meter_ids[i]]["x_i"]),
    )


group_id = "GROUP1"
establish_multicast_group(group_id, ["SM1", "SM2"], gateway_key)
for smart_meter_id in ["SM1", "SM2"]:
    encrypted_data = None
    for message in sent_messages:
        if message["smart_meter_id"] == smart_meter_id:
            if "encrypted_data" in message:
                encrypted_data = message["encrypted_data"]
                break
    if encrypted_data:
        message = {
            "smart_meter_id": smart_meter_id,
            "timestamp": get_timestamp(),
            "encrypted_data": encrypted_data,
        }
        join_multicast_group(message, gateway_key)

Sending message: {"smart_meter_id": "SM1", "h_sn": "33fe386e5930852f437d79a30856c8f5087b243f7daa391a0889f195a23aa9ff"}
Sending message: {"smart_meter_id": "SM2", "h_sn": "1235a7c46fb76cd78e631327e23f273e524db3c81f5e3bb1a8f2887f49fefe5e"}
Sending message: {"smart_meter_id": "SM3", "h_sn": "43d054a780a6416f01e81b2ba086b47933ebb67e36a1fbda52b509583595af81"}

Registering: SM1
 N_I: 52dea32d00677292468b23bb380236aa34da79a5130b2f04276ae8bba0814b5f
 X_I: 377fda7bf533e1fbf5a53a84a45041c0

Sending message: {"smart_meter_id": "SM1", "n_i": "52dea32d00677292468b23bb380236aa34da79a5130b2f04276ae8bba0814b5f", "x_i": "377fda7bf533e1fbf5a53a84a45041c0", "hh_sn_i": "e68967d75940ac9bd482480380431b9a5b9dd756ec5c2b3ccd5c386127da8634"}

Registering: SM2
 N_I: 8a2a13d7eac509bbf523d8330eb2352a0179c9c3c2d485d19f7dceb7c26c3b57
 X_I: f4e7cc0ced14b5ed3ee61a8aa215b0cb

Sending message: {"smart_meter_id": "SM2", "n_i": "8a2a13d7eac509bbf523d8330eb2352a0179c9c3c2d485d19f7dceb7c26c3b57", "x_i": "f4e7cc0ced14b5ed3ee